In [1]:
import pandas as pd 
import numpy as np 
import openai
import random

In [2]:
df = pd.read_csv('New_Cleaned_Engineer.csv')
df.head()

,RequisitionID,CleanedJobTitle,JobDescription
0,NaN,Licensed Stationary Engineer,Licensed Stationary Engineer Froedtert South K...
1,00000224907,Guidance Navigation Control GNC Engineer Lead,The Boeing Company search Lead Guidance Naviga...
2,00000331804,Propulsion Engineer Associate MidLevel Experie...,Job Description At Boeing innovate collaborate...
3,00000336462,Senior Process Controls Engineer,Job Description At Boeing innovate collaborate...
4,00000338951,RFMicrowave Engineer Level,Job Description At Boeing innovate collaborate...


In [3]:
df.drop(df.index[0],inplace = True)
df.reset_index(drop = True, inplace = True)
df.head()

,RequisitionID,CleanedJobTitle,JobDescription
0,00000224907,Guidance Navigation Control GNC Engineer Lead,The Boeing Company search Lead Guidance Naviga...
1,00000331804,Propulsion Engineer Associate MidLevel Experie...,Job Description At Boeing innovate collaborate...
2,00000336462,Senior Process Controls Engineer,Job Description At Boeing innovate collaborate...
3,00000338951,RFMicrowave Engineer Level,Job Description At Boeing innovate collaborate...
4,00000339390,Interior Stress Analysis Engineer Career Expert,Job Description At Boeing innovate collaborate...


In [4]:
openai.api_key = ""

In [5]:
# Randomly select 100 rows
sample_df = df.sample(n=100, random_state=42)
sample_df.head()

,RequisitionID,CleanedJobTitle,JobDescription
2408,143642,Information Security EngineerLevel Cloud,Requisition Number 143642 Job Description Cint...
19788,SENIO004432,Senior Engineer Electronics Commercial Develop...,Description Purpose Position Develop from conc...
16745,R-021199,Engineer DER Planning Hybrid,hybrid role . first three months fulltime offi...
9893,356-383,Cybersecurity Engineer,position support systems configuration within ...
8144,234532W,Fire Protection Engineer,DuPont working things matter whether providing...


In [6]:
import os

# Local CSV file path
csv_path = "C:\\Users\\13479\\Desktop\\Candogram Project\\"

# Extract directory from the CSV file's path
csv_directory = os.path.dirname(csv_path)

# Construct the path for the processed_ids.txt file
processed_ids_file = os.path.join(csv_directory, "processed_ids.txt")

In [7]:
def load_processed_ids():
    """Load processed RequisitionIDs from a file."""
    global processed_ids
    if os.path.exists(processed_ids_file):
        with open(processed_ids_file, 'r') as f:
            processed_ids = set(f.read().splitlines())
    else:
        processed_ids = set()

load_processed_ids()

In [44]:
def get_new_sample_without_update(data, n=50):
    """Fetch a new sample without updating the processed_ids set."""
    # Filter out rows with RequisitionIDs that have already been processed
    unprocessed_data = data[~data['RequisitionID'].isin(processed_ids)]
    # Randomly select rows from the unprocessed data
    return unprocessed_data.sample(n=n, random_state=42)

In [9]:
def save_processed_ids():
    """Save processed RequisitionIDs to a file."""
    global processed_ids
    with open(processed_ids_file, 'w') as f:
        for req_id in processed_ids:
            f.write(f"{req_id}\n")

In [10]:
def categorize_descriptions(job_description):
    # Define the user prompt message
    prompt = job_description
    
    # Create a chatbot using ChatCompletion.create() function
    completion = openai.ChatCompletion.create(
        # Use GPT 3.5 as the LLM
        model="gpt-3.5-turbo",
        # Pre-define conversation messages for the possible roles
        messages=[
            {"role": "system", "content": """
            I want you to act as a career counselor. I am going to give you a job description 
            that a student is interested in, and you will help them determine what part of the job description is marketing, 
            description, requirements, and legal. Marketing tends to talk about company culture, perks and benefits, growth 
            oppurtunities, exciting challenges, team and leadership, company achievements, and work life balance. The description 
            focuses on your role, responsibilities, and who you report to or who reports to you. Requirements talk about skills, 
            qualifications, and experience. Lastly, legal talks about equal oppurtunity statements, visa requirements, background 
            checks and contractual obligations. You should also conduct research into job postings similar as this, as that could 
            help you solve this task. My first request is to return every sentence labeled in one of the 4 categories given and return 
            it in JSON format. We want to focus on making sure that we are not mislabeling any part of the description as marketing, 
            requirements, or legals, and it is more acceptable to have more then we need in the description, then be missing out on 
            important descriptions for the job. Each sentence should be labeled only once. There shouldn't be repeated sentences returned. 
            However, every sentence from the original job description needs to be accounted for. If there's any mentions of numbers, 
            it probably should be in the requirements section.
            """},
            {"role": "user", "content": prompt}
        ]
    )
    
    # Return the output from the LLM model
    return completion.choices[0].message['content']

In [78]:
if 'sample_df' not in locals():
    sample_df = pd.DataFrame()
# Modified workflow

# Step 1: Fetch a new sample without updating processed_ids
new_sample = get_new_sample_without_update(df)

# Step 2: Process the new sample
new_sample["EditedJobDescription"] = new_sample["JobDescription"].apply(categorize_descriptions)

# Step 3: Append the processed sample to sample_df
sample_df = pd.concat([sample_df, new_sample], ignore_index=True)

# Step 4: Update processed_ids with the IDs of the processed sample
processed_ids = set(processed_ids)
processed_ids.update(new_sample['RequisitionID'].values)

# Step 5: Save the updated set of processed RequisitionIDs
save_processed_ids()

sample_df

,RequisitionID,CleanedJobTitle,JobDescription,EditedJobDescription
0,143642,Information Security EngineerLevel Cloud,Requisition Number 143642 Job Description Cint...,"{\n ""marketing"": [\n ""Cintas offers compre..."
1,SENIO004432,Senior Engineer Electronics Commercial Develop...,Description Purpose Position Develop from conc...,"{\n ""marketing"": [\n ""Develop from concept..."
2,R-021199,Engineer DER Planning Hybrid,hybrid role . first three months fulltime offi...,"{\n ""marketing"": [\n ""largest Energy..."
3,356-383,Cybersecurity Engineer,position support systems configuration within ...,"{\n ""marketing"": [\n ""Support Cybersecurit..."
4,234532W,Fire Protection Engineer,DuPont working things matter whether providing...,"{\n ""marketing"": [\n ""Get know Purpose mak..."
...,...,...,...,...
495,01.01.16-2023-21,Computer Engineer,Dynetics wholly owned subsidiary Leidos seekin...,"{\n ""marketing"": [],\n ""description"": [\n ..."
496,334701,Multivendor Customer Service Engineer MIUS Phi...,Nows time inspire future healthcaretogether. A...,"{\n ""marketing"": [\n ""At Siemens Healthine..."
497,R2028871,Engineer,Location Four Houston Center 1200 McKinney Sui...,"{\n ""marketing"":[\n ""Location Four Hous..."
498,2023-30496,Engineer,Pyramid Global Hospitality people come first. ...,"{\n ""marketing"": [\n ""Pyramid Global Hospi..."


In [79]:
sample_df['EditedJobDescription'].isna().sum()

0

In [80]:
sample_df[sample_df['EditedJobDescription'].isna()]

,RequisitionID,CleanedJobTitle,JobDescription,EditedJobDescription


In [81]:
sample_df.dropna(subset = ['EditedJobDescription'], inplace = True)
sample_df

,RequisitionID,CleanedJobTitle,JobDescription,EditedJobDescription
0,143642,Information Security EngineerLevel Cloud,Requisition Number 143642 Job Description Cint...,"{\n ""marketing"": [\n ""Cintas offers compre..."
1,SENIO004432,Senior Engineer Electronics Commercial Develop...,Description Purpose Position Develop from conc...,"{\n ""marketing"": [\n ""Develop from concept..."
2,R-021199,Engineer DER Planning Hybrid,hybrid role . first three months fulltime offi...,"{\n ""marketing"": [\n ""largest Energy..."
3,356-383,Cybersecurity Engineer,position support systems configuration within ...,"{\n ""marketing"": [\n ""Support Cybersecurit..."
4,234532W,Fire Protection Engineer,DuPont working things matter whether providing...,"{\n ""marketing"": [\n ""Get know Purpose mak..."
...,...,...,...,...
495,01.01.16-2023-21,Computer Engineer,Dynetics wholly owned subsidiary Leidos seekin...,"{\n ""marketing"": [],\n ""description"": [\n ..."
496,334701,Multivendor Customer Service Engineer MIUS Phi...,Nows time inspire future healthcaretogether. A...,"{\n ""marketing"": [\n ""At Siemens Healthine..."
497,R2028871,Engineer,Location Four Houston Center 1200 McKinney Sui...,"{\n ""marketing"":[\n ""Location Four Hous..."
498,2023-30496,Engineer,Pyramid Global Hospitality people come first. ...,"{\n ""marketing"": [\n ""Pyramid Global Hospi..."


In [82]:
sample_df['RequisitionID'].nunique()

500

In [83]:
sample_df = sample_df.drop_duplicates(subset = 'RequisitionID', keep = 'first')
sample_df

,RequisitionID,CleanedJobTitle,JobDescription,EditedJobDescription
0,143642,Information Security EngineerLevel Cloud,Requisition Number 143642 Job Description Cint...,"{\n ""marketing"": [\n ""Cintas offers compre..."
1,SENIO004432,Senior Engineer Electronics Commercial Develop...,Description Purpose Position Develop from conc...,"{\n ""marketing"": [\n ""Develop from concept..."
2,R-021199,Engineer DER Planning Hybrid,hybrid role . first three months fulltime offi...,"{\n ""marketing"": [\n ""largest Energy..."
3,356-383,Cybersecurity Engineer,position support systems configuration within ...,"{\n ""marketing"": [\n ""Support Cybersecurit..."
4,234532W,Fire Protection Engineer,DuPont working things matter whether providing...,"{\n ""marketing"": [\n ""Get know Purpose mak..."
...,...,...,...,...
495,01.01.16-2023-21,Computer Engineer,Dynetics wholly owned subsidiary Leidos seekin...,"{\n ""marketing"": [],\n ""description"": [\n ..."
496,334701,Multivendor Customer Service Engineer MIUS Phi...,Nows time inspire future healthcaretogether. A...,"{\n ""marketing"": [\n ""At Siemens Healthine..."
497,R2028871,Engineer,Location Four Houston Center 1200 McKinney Sui...,"{\n ""marketing"":[\n ""Location Four Hous..."
498,2023-30496,Engineer,Pyramid Global Hospitality people come first. ...,"{\n ""marketing"": [\n ""Pyramid Global Hospi..."


In [84]:
with open("processed_ids.txt", "r") as file:
    processed_ids = [line.strip() for line in file.readlines()]
df_ids = sample_df["RequisitionID"].tolist()
are_equal = set(df_ids) == set(processed_ids)
are_equal

True

In [88]:
sample_df.to_csv("Concatenated_Job_Description.csv", index = False)

In [86]:
new_df = pd.read_csv("Concatenated_Job_Description.csv")
new_df

,RequisitionID,CleanedJobTitle,JobDescription,EditedJobDescription
0,143642,Information Security EngineerLevel Cloud,Requisition Number 143642 Job Description Cint...,"{\n ""marketing"": [\n ""Cintas offers compre..."
1,SENIO004432,Senior Engineer Electronics Commercial Develop...,Description Purpose Position Develop from conc...,"{\n ""marketing"": [\n ""Develop from concept..."
2,R-021199,Engineer DER Planning Hybrid,hybrid role . first three months fulltime offi...,"{\n ""marketing"": [\n ""largest Energy..."
3,356-383,Cybersecurity Engineer,position support systems configuration within ...,"{\n ""marketing"": [\n ""Support Cybersecurit..."
4,234532W,Fire Protection Engineer,DuPont working things matter whether providing...,"{\n ""marketing"": [\n ""Get know Purpose mak..."
...,...,...,...,...
495,01.01.16-2023-21,Computer Engineer,Dynetics wholly owned subsidiary Leidos seekin...,"{\n ""marketing"": [],\n ""description"": [\n ..."
496,334701,Multivendor Customer Service Engineer MIUS Phi...,Nows time inspire future healthcaretogether. A...,"{\n ""marketing"": [\n ""At Siemens Healthine..."
497,R2028871,Engineer,Location Four Houston Center 1200 McKinney Sui...,"{\n ""marketing"":[\n ""Location Four Hous..."
498,2023-30496,Engineer,Pyramid Global Hospitality people come first. ...,"{\n ""marketing"": [\n ""Pyramid Global Hospi..."


In [87]:
random_job_description = sample_df.sample(n=1)['EditedJobDescription']
pd.set_option('display.max_colwidth', None)
random_job_description

307    {\n  "marketing": [\n    "Many worlds top engineers scientists come together",\n    "Water business view communitys interaction water bit differentlyas single holistic system rather unconnected networks divided jurisdictional boundaries.",\n    "Working throughout hydrologic cycle delivering sustainable solutions make sure water available needed.",\n    "experts guide work scientific rigor innovative spirit vision growth.",\n    "Every day help communities improve reuse protect precious resource future generations.",\n    "2025 were hiring 2,000 people like you join team.",\n    "Grow us H2OU.",\n    "Your Opportunity",\n    "We looking talented Water Resources EngineerinTraining EIT wants part purposedriven organization thats focused helping clients colleagues communities thrive.",\n    "people Stantecs valuable resource joining team able leverage career experiences expertise within culture values inclusion celebrates shared success applauds ambition.",\n    "Stantecs Water Res

In [50]:
pd.reset_option('display.max_colwidth')